In [14]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [23]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[1])

Training entries: 25000, labels: 25000
[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


In [24]:
word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print (decode_review(train_data[1]))

<START> big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal <UNK> the hair is big lots of boobs <UNK> men wear those cut <UNK> shirts that show off their <UNK> sickening that men actually wore them and the music is just <UNK> trash that plays over and over again in almost every scene there is trashy music boobs and <UNK> taking away bodies and the gym still doesn't close for <UNK> all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then


In [17]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding='post', maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding='post', maxlen=256)

In [18]:
vocab_size = 10000

model = keras.Sequential([keras.layers.Embedding(vocab_size, 16), keras.layers.GlobalAveragePooling1D(), keras.layers.Dense(16, activation=tf.nn.relu), keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [19]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [21]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 3s 215us/sample - loss: 0.6920 - acc: 0.5765 - val_loss: 0.6902 - val_acc: 0.6196
Epoch 2/40
15000/15000 [==============================] - 3s 168us/sample - loss: 0.6866 - acc: 0.7067 - val_loss: 0.6825 - val_acc: 0.7462
Epoch 3/40
15000/15000 [==============================] - 3s 167us/sample - loss: 0.6741 - acc: 0.7605 - val_loss: 0.6665 - val_acc: 0.7561
Epoch 4/40
15000/15000 [==============================] - 3s 169us/sample - loss: 0.6511 - acc: 0.7715 - val_loss: 0.6396 - val_acc: 0.7554
Epoch 5/40
15000/15000 [==============================] - 2s 165us/sample - loss: 0.6160 - acc: 0.7969 - val_loss: 0.6027 - val_acc: 0.7845
Epoch 6/40
15000/15000 [==============================] - 3s 173us/sample - loss: 0.5712 - acc: 0.8159 - val_loss: 0.5595 - val_acc: 0.8064
Epoch 7/40
15000/15000 [==============================] - 3s 169us/sample - loss: 0.5211 - acc: 0.8325 - val_l

In [22]:
results = model.evaluate(test_data, test_labels)
print(results)

25000/25000 [==============================] - 1s 54us/sample - loss: 0.3354 - acc: 0.8711
[0.33539290192604065, 0.87112]
